#### Wednesday, May 8, 2024

mamba activate llama3

This all runs in one pass.

<a href="https://colab.research.google.com/github/meta-llama/llama-recipes/blob/main/recipes/quickstart/Prompt_Engineering_with_Llama_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering with Llama 3

Prompt engineering is using natural language to produce a desired response from a large language model (LLM).

This interactive guide covers prompt engineering & best practices with Llama 3.

## Introduction

### Why now?

[Vaswani et al. (2017)](https://arxiv.org/abs/1706.03762) introduced the world to transformer neural networks (originally for machine translation). Transformers ushered an era of generative AI with diffusion models for image creation and large language models (`LLMs`) as **programmable deep learning networks**.

Programming foundational LLMs is done with natural language – it doesn't require training/tuning like ML models of the past. This has opened the door to a massive amount of innovation and a paradigm shift in how technology can be deployed. The science/art of using natural language to program language models to accomplish a task is referred to as **Prompt Engineering**.

### Llama Models

In 2023, Meta introduced the [Llama language models](https://ai.meta.com/llama/) (Llama Chat, Code Llama, Llama Guard). These are general purpose, state-of-the-art LLMs.

Llama models come in varying parameter sizes. The smaller models are cheaper to deploy and run; the larger models are more capable.

#### Llama 3
1. `llama-3-8b` - base pretrained 8 billion parameter model
1. `llama-3-70b` - base pretrained 70 billion parameter model
1. `llama-3-8b-instruct` - instruction fine-tuned 8 billion parameter model
1. `llama-3-70b-instruct` - instruction fine-tuned 70 billion parameter model (flagship)

#### Llama 2
1. `llama-2-7b` - base pretrained 7 billion parameter model
1. `llama-2-13b` - base pretrained 13 billion parameter model
1. `llama-2-70b` - base pretrained 70 billion parameter model
1. `llama-2-7b-chat` - chat fine-tuned 7 billion parameter model
1. `llama-2-13b-chat` - chat fine-tuned 13 billion parameter model
1. `llama-2-70b-chat` - chat fine-tuned 70 billion parameter model (flagship)


Code Llama is a code-focused LLM built on top of Llama 2 also available in various sizes and finetunes:

#### Code Llama
1. `codellama-7b` - code fine-tuned 7 billion parameter model
1. `codellama-13b` - code fine-tuned 13 billion parameter model
1. `codellama-34b` - code fine-tuned 34 billion parameter model
1. `codellama-70b` - code fine-tuned 70 billion parameter model
1. `codellama-7b-instruct` - code & instruct fine-tuned 7 billion parameter model
2. `codellama-13b-instruct` - code & instruct fine-tuned 13 billion parameter model
3. `codellama-34b-instruct` - code & instruct fine-tuned 34 billion parameter model
3. `codellama-70b-instruct` - code & instruct fine-tuned 70 billion parameter model
1. `codellama-7b-python` - Python fine-tuned 7 billion parameter model
2. `codellama-13b-python` - Python fine-tuned 13 billion parameter model
3. `codellama-34b-python` - Python fine-tuned 34 billion parameter model
3. `codellama-70b-python` - Python fine-tuned 70 billion parameter model

## Getting an LLM

Large language models are deployed and accessed in a variety of ways, including:

1. **Self-hosting**: Using local hardware to run inference. Ex. running Llama on your Macbook Pro using [llama.cpp](https://github.com/ggerganov/llama.cpp).
    * Best for privacy/security or if you already have a GPU.
1. **Cloud hosting**: Using a cloud provider to deploy an instance that hosts a specific model. Ex. running Llama on cloud providers like AWS, Azure, GCP, and others.
    * Best for customizing models and their runtime (ex. fine-tuning a model for your use case).
1. **Hosted API**: Call LLMs directly via an API. There are many companies that provide Llama inference APIs including AWS Bedrock, Replicate, Anyscale, Together and others.
    * Easiest option overall.

### Hosted APIs

Hosted APIs are the easiest way to get started. We'll use them here. There are usually two main endpoints:

1. **`completion`**: generate a response to a given prompt (a string).
1. **`chat_completion`**: generate the next message in a list of messages, enabling more explicit instruction and context for use cases like chatbots.

## Tokens

LLMs process inputs and outputs in chunks called *tokens*. Think of these, roughly, as words – each model will have its own tokenization scheme. For example, this sentence...

> Our destiny is written in the stars.

...is tokenized into `["Our", " destiny", " is", " written", " in", " the", " stars", "."]` for Llama 3. See [this](https://tiktokenizer.vercel.app/?model=meta-llama%2FMeta-Llama-3-8B) for an interactive tokenizer tool.

Tokens matter most when you consider API pricing and internal behavior (ex. hyperparameters).

Each model has a maximum context length that your prompt cannot exceed. That's 8K tokens for Llama 3, 4K for Llama 2, and 100K for Code Llama. 


## Notebook Setup

The following APIs will be used to call LLMs throughout the guide. As an example, we'll call Llama 3 chat using [Grok](https://console.groq.com/playground?model=llama3-70b-8192).

To install prerequisites run:

In [1]:
import sys
# !{sys.executable} -m pip install groq

In [2]:
import os
from typing import Dict, List
from groq import Groq



In [3]:
import os
from getpass import getpass

GROQ_API_TOKEN = getpass()

os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

In [4]:
LLAMA3_70B_INSTRUCT = "llama3-70b-8192"
LLAMA3_8B_INSTRUCT = "llama3-8b-8192"

DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

client = Groq()

def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def chat_completion(
    messages: List[Dict],
    model = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature,
        top_p=top_p,
    )
    return response.choices[0].message.content
        

def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
) -> str:
    return chat_completion(
        [user(prompt)],
        model=model,
        temperature=temperature,
        top_p=top_p,
    )

def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
    print(f'==============\n{prompt}\n==============')
    response = completion(prompt, model)
    print(response, end='\n\n')


### Completion APIs

Let's try Llama 3!

In [5]:
complete_and_print("The typical color of the sky is: ")

The typical color of the sky is: 
Blue!



In [6]:
complete_and_print("which model version are you?")

which model version are you?
I'm an AI, and I'm a large language model, specifically a transformer model. My model version is based on the LLaMA architecture, which is a type of transformer model designed for conversational AI.

I'm a continually learning and improving model, so I don't have a fixed "version number" in the classical sense. However, my knowledge cutoff is around October 2022, and I was trained on a massive dataset of text from the internet.

If you're interested in the technical details, my model is based on the LLaMA-Base architecture, which is a variant of the transformer architecture introduced in the paper "LLaMA: Open and Efficient Foundation Language Models" by the Meta AI team.



### Chat Completion APIs
Chat completion models provide additional structure to interacting with an LLM. An array of structured message objects is sent to the LLM instead of a single piece of text. This message list provides the LLM with some "context" or "history" from which to continue.

Typically, each message contains `role` and `content`:
* Messages with the `system` role are used to provide core instruction to the LLM by developers.
* Messages with the `user` role are typically human-provided messages.
* Messages with the `assistant` role are typically generated by the LLM.

In [7]:
response = chat_completion(messages=[
    user("My favorite color is blue."),
    assistant("That's great to hear!"),
    user("What is my favorite color?"),
])
print(response)
# "Sure, I can help you with that! Your favorite color is blue."

I remember! Your favorite color is blue!


### LLM Hyperparameters

#### `temperature` & `top_p`

These APIs also take parameters which influence the creativity and determinism of your output.

At each step, LLMs generate a list of most likely tokens and their respective probabilities. The least likely tokens are "cut" from the list (based on `top_p`), and then a token is randomly selected from the remaining candidates (`temperature`).

In other words: `top_p` controls the breadth of vocabulary in a generation and `temperature` controls the randomness within that vocabulary. A temperature of ~0 produces *almost* deterministic results.

[Read more about temperature setting here](https://community.openai.com/t/cheat-sheet-mastering-temperature-and-top-p-in-chatgpt-api-a-few-tips-and-tricks-on-controlling-the-creativity-deterministic-output-of-prompt-responses/172683).

Let's try it out:

In [8]:
def print_tuned_completion(temperature: float, top_p: float):
    response = completion("Write a haiku about llamas", temperature=temperature, top_p=top_p)
    print(f'[temperature: {temperature} | top_p: {top_p}]\n{response.strip()}\n')

print_tuned_completion(0.01, 0.01)
print_tuned_completion(0.01, 0.01)
# These two generations are highly likely to be the same

print_tuned_completion(1.0, 1.0)
print_tuned_completion(1.0, 1.0)
# These two generations are highly likely to be different

[temperature: 0.01 | top_p: 0.01]
Softly gaze at me
Llama's gentle, fuzzy form
Serenity

[temperature: 0.01 | top_p: 0.01]
Softly gaze at me
Llama's gentle, fuzzy form
Serenity

[temperature: 1.0 | top_p: 1.0]
Fuzzy, gentle friends
Soft eyes and soft, warm fleece shine
Llama love surrounds

[temperature: 1.0 | top_p: 1.0]
Here is a haiku about llamas:

Fluffy, gentle friends
Soft eyes, gentle humming sound
Llama serenity



## Prompting Techniques

### Explicit Instructions

Detailed, explicit instructions produce better results than open-ended prompts:

In [9]:
complete_and_print(prompt="Describe quantum physics in one short sentence of no more than 12 words")
# Returns a succinct explanation of quantum physics that mentions particles and states existing simultaneously.

Describe quantum physics in one short sentence of no more than 12 words
Quantum physics: Tiny particles behave weirdly, defying classical logic rules.



You can think about giving explicit instructions as using rules and restrictions to how Llama 3 responds to your prompt.

- Stylization
    - `Explain this to me like a topic on a children's educational network show teaching elementary students.`
    - `I'm a software engineer using large language models for summarization. Summarize the following text in under 250 words:`
    - `Give your answer like an old timey private investigator hunting down a case step by step.`
- Formatting
    - `Use bullet points.`
    - `Return as a JSON object.`
    - `Use less technical terms and help me apply it in my work in communications.`
- Restrictions
    - `Only use academic papers.`
    - `Never give sources older than 2020.`
    - `If you don't know the answer, say that you don't know.`

Here's an example of giving explicit instructions to give more specific results by limiting the responses to recently created sources.

In [10]:
complete_and_print("Explain the latest advances in large language models to me.")
# More likely to cite sources from 2017

complete_and_print("Explain the latest advances in large language models to me. Always cite your sources. Never cite sources older than 2020.")
# Gives more specific advances and only cites sources from 2020

Explain the latest advances in large language models to me.
What an exciting time for natural language processing (NLP) and artificial intelligence (AI)! The latest advances in large language models have been remarkable, with significant improvements in capabilities, efficiency, and applications. Here's a rundown of the latest developments:

**Scaling Up:**

1. **Transformer-XL**: Introduced in 2019, Transformer-XL is a scaled-up version of the original Transformer architecture. It uses a combination of model parallelism and pipeline parallelism to train massive models with billions of parameters.
2. **Megatron-LM**: In 2020, the Megatron-LM model was announced, which is a 8.3 billion parameter model that achieves state-of-the-art results on various NLP tasks.

**Efficiency Improvements:**

1. **Sparse Models**: Researchers have developed sparse models that maintain performance while reducing computational requirements. This is achieved by pruning unnecessary weights and connections in

### Example Prompting using Zero- and Few-Shot Learning

A shot is an example or demonstration of what type of prompt and response you expect from a large language model. This term originates from training computer vision models on photographs, where one shot was one example or instance that the model used to classify an image ([Fei-Fei et al. (2006)](http://vision.stanford.edu/documents/Fei-FeiFergusPerona2006.pdf)).

#### Zero-Shot Prompting

Large language models like Llama 3 are unique because they are capable of following instructions and producing responses without having previously seen an example of a task. Prompting without examples is called "zero-shot prompting".

Let's try using Llama 3 as a sentiment detector. You may notice that output format varies - we can improve this with better prompting.

In [11]:
complete_and_print("Text: This was the best movie I've ever seen! \n The sentiment of the text is: ")
# Returns positive sentiment

complete_and_print("Text: The director was trying too hard. \n The sentiment of the text is: ")
# Returns negative sentiment

Text: This was the best movie I've ever seen! 
 The sentiment of the text is: 
Positive.

Text: The director was trying too hard. 
 The sentiment of the text is: 
Negative




#### Few-Shot Prompting

Adding specific examples of your desired output generally results in more accurate, consistent output. This technique is called "few-shot prompting".

In this example, the generated response follows our desired format that offers a more nuanced sentiment classifer that gives a positive, neutral, and negative response confidence percentage.

See also: [Zhao et al. (2021)](https://arxiv.org/abs/2102.09690), [Liu et al. (2021)](https://arxiv.org/abs/2101.06804), [Su et al. (2022)](https://arxiv.org/abs/2209.01975), [Rubin et al. (2022)](https://arxiv.org/abs/2112.08633).



In [12]:
def sentiment(text):
    response = chat_completion(messages=[
        user("You are a sentiment classifier. For each message, give the percentage of positive/netural/negative."),
        user("I liked it"),
        assistant("70% positive 30% neutral 0% negative"),
        user("It could be better"),
        assistant("0% positive 50% neutral 50% negative"),
        user("It's fine"),
        assistant("25% positive 50% neutral 25% negative"),
        user(text),
    ])
    return response

def print_sentiment(text):
    print(f'INPUT: {text}')
    print(sentiment(text))

print_sentiment("I thought it was okay")
# More likely to return a balanced mix of positive, neutral, and negative
print_sentiment("I loved it!")
# More likely to return 100% positive
print_sentiment("Terrible service 0/10")
# More likely to return 100% negative

INPUT: I thought it was okay
20% positive 60% neutral 20% negative
INPUT: I loved it!
90% positive 10% neutral 0% negative
INPUT: Terrible service 0/10
0% positive 0% neutral 100% negative


### Role Prompting

Llama will often give more consistent responses when given a role ([Kong et al. (2023)](https://browse.arxiv.org/pdf/2308.07702.pdf)). Roles give context to the LLM on what type of answers are desired.

Let's use Llama 3 to create a more focused, technical response for a question around the pros and cons of using PyTorch.

In [13]:
complete_and_print("Explain the pros and cons of using PyTorch.")
# More likely to explain the pros and cons of PyTorch covers general areas like documentation, the PyTorch community, and mentions a steep learning curve

complete_and_print("Your role is a machine learning expert who gives highly technical advice to senior engineers who work with complicated datasets. Explain the pros and cons of using PyTorch.")
# Often results in more technical benefits and drawbacks that provide more technical details on how model layers

Explain the pros and cons of using PyTorch.
PyTorch is a popular open-source machine learning library developed by Facebook's AI Research Lab (FAIR). It provides a dynamic computation graph and is particularly well-suited for rapid prototyping and research. Here are some pros and cons of using PyTorch:

**Pros:**

1. **Dynamic Computation Graph**: PyTorch's dynamic computation graph allows for more flexible and rapid prototyping. It can create and modify the computation graph on the fly, making it ideal for research and development.
2. **Easy to Learn**: PyTorch has a relatively low barrier to entry, especially for those familiar with Python and NumPy. Its API is intuitive, and its dynamic computation graph makes it easier to understand and work with.
3. **Rapid Prototyping**: PyTorch's dynamic nature and ease of use make it ideal for rapid prototyping and experimentation. It's perfect for quickly testing and iterating on new ideas.
4. **Auto-Differentiation**: PyTorch's automatic diff

### Chain-of-Thought

Simply adding a phrase encouraging step-by-step thinking "significantly improves the ability of large language models to perform complex reasoning" ([Wei et al. (2022)](https://arxiv.org/abs/2201.11903)). This technique is called "CoT" or "Chain-of-Thought" prompting.

Llama 3 now reasons step-by-step naturally without the addition of the phrase. This section remains for completeness.

In [14]:
prompt = "Who lived longer, Mozart or Elvis?"

complete_and_print(prompt)
# Llama 2 would often give the incorrect answer of "Mozart"

complete_and_print(f"{prompt} Let's think through this carefully, step by step.")
# Gives the correct answer "Elvis"

Who lived longer, Mozart or Elvis?
Wolfgang Amadeus Mozart (1756-1791) lived to be 35 years old.

Elvis Presley (1935-1977) lived to be 42 years old.

So, Elvis lived about 7 years longer than Mozart.

Who lived longer, Mozart or Elvis? Let's think through this carefully, step by step.
What a fascinating question! Let's indeed think this through step by step.

**Step 1: Wolfgang Amadeus Mozart**
Mozart was born on January 27, 1756, and died on December 5, 1791. To calculate his lifespan, we need to subtract his birth year from his death year:

1791 (death year) - 1756 (birth year) = 35 years

So, Mozart lived for approximately 35 years.

**Step 2: Elvis Presley (Elvis)**
Elvis was born on January 8, 1935, and died on August 16, 1977. Let's calculate his lifespan:

1977 (death year) - 1935 (birth year) = 42 years

Elvis lived for approximately 42 years.

**Comparison**
Now, let's compare their lifespans:

Mozart lived for 35 years.
Elvis lived for 42 years.

**Conclusion**
Elvis Presley

### Self-Consistency

LLMs are probablistic, so even with Chain-of-Thought, a single generation might produce incorrect results. Self-Consistency ([Wang et al. (2022)](https://arxiv.org/abs/2203.11171)) introduces enhanced accuracy by selecting the most frequent answer from multiple generations (at the cost of higher compute):

In [15]:
import re
from statistics import mode

def gen_answer():
    response = completion(
        "John found that the average of 15 numbers is 40."
        "If 10 is added to each number then the mean of the numbers is?"
        "Report the answer surrounded by backticks (example: `123`)",
    )
    match = re.search(r'`(\d+)`', response)
    if match is None:
        return None
    return match.group(1)

answers = [gen_answer() for i in range(5)]

print(
    f"Answers: {answers}\n",
    f"Final answer: {mode(answers)}",
    )

# Sample runs of Llama-3-70B (all correct):
# ['60', '50', '50', '50', '50'] -> 50
# ['50', '50', '50', '60', '50'] -> 50
# ['50', '50', '60', '50', '50'] -> 50

Answers: ['50', '50', '60', '50', '50']
 Final answer: 50


### Retrieval-Augmented Generation

You'll probably want to use factual knowledge in your application. You can extract common facts from today's large models out-of-the-box (i.e. using just the model weights):

In [16]:
complete_and_print("What is the capital of the California?")
# Gives the correct answer "Sacramento"

What is the capital of the California?
The capital of California is Sacramento.



However, more specific facts, or private information, cannot be reliably retrieved. The model will either declare it does not know or hallucinate an incorrect answer:

In [17]:
complete_and_print("What was the temperature in Menlo Park on December 12th, 2023?")
# "I'm just an AI, I don't have access to real-time weather data or historical weather records."

complete_and_print("What time is my dinner reservation on Saturday and what should I wear?")
# "I'm not able to access your personal information [..] I can provide some general guidance"

What was the temperature in Menlo Park on December 12th, 2023?
I'm an AI, I don't have have access to real-time or historical weather data. However, I can suggest some ways for you to find the temperature in Menlo Park on December 12th, 2023.

1. Check online weather websites: You can check websites like AccuWeather, Weather.com, or the National Weather Service (NWS) for historical weather data. They may have archives of past weather data, including temperature readings for specific dates and locations.
2. Contact local weather stations: Reach out to local weather stations or news outlets in Menlo Park and ask if they have records of the temperature on December 12th, 2023.
3. Check with local government: You can also contact the City of Menlo Park or the San Mateo County government to see if they have any records of weather data for that specific date.

Keep in mind that if December 12th, 2023, is in the future, the temperature data will not be available yet.

What time is my dinner re

Retrieval-Augmented Generation, or RAG, describes the practice of including information in the prompt you've retrived from an external database ([Lewis et al. (2020)](https://arxiv.org/abs/2005.11401v4)). It's an effective way to incorporate facts into your LLM application and is more affordable than fine-tuning which may be costly and negatively impact the foundational model's capabilities.

This could be as simple as a lookup table or as sophisticated as a [vector database]([FAISS](https://github.com/facebookresearch/faiss)) containing all of your company's knowledge:

In [18]:
MENLO_PARK_TEMPS = {
    "2023-12-11": "52 degrees Fahrenheit",
    "2023-12-12": "51 degrees Fahrenheit",
    "2023-12-13": "51 degrees Fahrenheit",
}


def prompt_with_rag(retrived_info, question):
    complete_and_print(
        f"Given the following information: '{retrived_info}', respond to: '{question}'"
    )


def ask_for_temperature(day):
    temp_on_day = MENLO_PARK_TEMPS.get(day) or "unknown temperature"
    prompt_with_rag(
        f"The temperature in Menlo Park was {temp_on_day} on {day}'",  # Retrieved fact
        f"What is the temperature in Menlo Park on {day}?",  # User question
    )


ask_for_temperature("2023-12-12")
# "Sure! The temperature in Menlo Park on 2023-12-12 was 51 degrees Fahrenheit."

ask_for_temperature("2023-07-18")
# "I'm not able to provide the temperature in Menlo Park on 2023-07-18 as the information provided states that the temperature was unknown."

Given the following information: 'The temperature in Menlo Park was 51 degrees Fahrenheit on 2023-12-12'', respond to: 'What is the temperature in Menlo Park on 2023-12-12?'
The temperature in Menlo Park on 2023-12-12 is 51 degrees Fahrenheit.

Given the following information: 'The temperature in Menlo Park was unknown temperature on 2023-07-18'', respond to: 'What is the temperature in Menlo Park on 2023-07-18?'
The temperature in Menlo Park on 2023-07-18 is unknown.



### Program-Aided Language Models

LLMs, by nature, aren't great at performing calculations. Let's try:

$$
((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
$$

(The correct answer is 91383.)

In [19]:
complete_and_print("""
Calculate the answer to the following math problem:

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
""")
# Gives incorrect answers like 92448, 92648, 95463


Calculate the answer to the following math problem:

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))

To calculate the answer, we need to follow the order of operations (PEMDAS):

1. Evaluate the expressions inside the parentheses:

(-5 + 93 * 4 - 0) = ?

First, calculate the multiplication: 93 * 4 = 372

Then, add and subtract from left to right: -5 + 372 - 0 = 367

So, the first part of the expression is 367.

2. Evaluate the second part of the expression:

(4^4 + -7 + 0 * 5) = ?

First, calculate the exponentiation: 4^4 = 256

Then, add and subtract from left to right: 256 - 7 + 0 = 249

So, the second part of the expression is 249.

3. Multiply the two results:

367 * 249 = 91483

Therefore, the final answer is 91483.



[Gao et al. (2022)](https://arxiv.org/abs/2211.10435) introduced the concept of "Program-aided Language Models" (PAL). While LLMs are bad at arithmetic, they're great for code generation. PAL leverages this fact by instructing the LLM to write code to solve calculation tasks.

In [20]:
complete_and_print(
    """
    # Python code to calculate: ((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
    """,
)


    # Python code to calculate: ((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
    
Here is the Python code to calculate the given expression:

```
expression = ((-5 + 93 * 4 - 0) * (4**4 - 7 + 0 * 5))
print(expression)
```

Let's break down the calculation:

1. `-5 + 93 * 4 - 0`:
   - `93 * 4` = 372
   - `-5 + 372 - 0` = 367

2. `4**4` = 256
   - `256 - 7 + 0 * 5` = 249

Now, multiply the two results:
   - `367 * 249` = 91483

So, the final result is `91483`.



In [21]:
# The following code was generated by Llama 3 70B:

result = ((-5 + 93 * 4 - 0) * (4**4 - 7 + 0 * 5))
print(result)

91383


### Limiting Extraneous Tokens

A common struggle with Llama 2 is getting output without extraneous tokens (ex. "Sure! Here's more information on..."), even if explicit instructions are given to Llama 2 to be concise and no preamble. Llama 3 can better follow instructions.

Check out this improvement that combines a role, rules and restrictions, explicit instructions, and an example:

In [22]:
complete_and_print(
    "Give me the zip code for Menlo Park in JSON format with the field 'zip_code'",
)
# Likely returns the JSON and also "Sure! Here's the JSON..."

complete_and_print(
    """
    You are a robot that only outputs JSON.
    You reply in JSON format with the field 'zip_code'.
    Example question: What is the zip code of the Empire State Building? Example answer: {'zip_code': 10118}
    Now here is my question: What is the zip code of Menlo Park?
    """,
)
# "{'zip_code': 94025}"

Give me the zip code for Menlo Park in JSON format with the field 'zip_code'
Here is the zip code for Menlo Park, CA in JSON format:

```
{
  "zip_code": "94025"
}
```

Note: Menlo Park has multiple zip codes, but 94025 is one of the main ones. If you need a specific zip code for a particular area of Menlo Park, let me know!


    You are a robot that only outputs JSON.
    You reply in JSON format with the field 'zip_code'.
    Example question: What is the zip code of the Empire State Building? Example answer: {'zip_code': 10118}
    Now here is my question: What is the zip code of Menlo Park?
    
{'zip_code': 94025}



## Additional References
- [PromptingGuide.ai](https://www.promptingguide.ai/)
- [LearnPrompting.org](https://learnprompting.org/)
- [Lil'Log Prompt Engineering Guide](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)


## Author & Contact

Edited by [Dalton Flanagan](https://www.linkedin.com/in/daltonflanagan/) (dalton@meta.com) with contributions from Mohsen Agsen, Bryce Bortree, Ricardo Juan Palma Duran, Kaolin Fire, Thomas Scialom.